In [1]:
from itertools import product
from tqdm import tqdm
import csv
from data_generation import word, chenyu
import opencc
import os


2023-01-24 11:12:52.812289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/yuan/anaconda3/envs/wtcpt/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/yuan/anaconda3/envs/wtcpt/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2023-01-24 11:12:54.891119: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-24 11:12:5

Word utils

In [15]:
sents_file = 'sents.csv'
sents_ = []
with open(sents_file,'r') as f:
    sents_ = [i for i in csv.reader(f)][1:]

sents = sents_[:200000]

In [ ]:
def get_len(a):
    return(len(a))

def word_data(sents:list, len:list, p:list, pos_tag:str):
    file_name = 'word_'+str(get_len(sents))
    if not os.path.isdir(file_name):
        os.mkdir(file_name)

    sents_ = []
    mask = []
    label = []

    for sent in tqdm(sents):
        try:
            sent = [sent[:max(len)]]
            a, b, c = word(sent, pos_tag, p)
            label.append([a,b])
            mask += c
            del a
            del b
            del c
        except:
            pass
    
    # save dict
    label_dict = {}
    for r in range(get_len(p)):
        for y in len:
            label_dict['label_r_p'.replace('r',str(p[r])).replace('p',str(y))] = []

    for i in range(get_len(sents)):
        for r in range(get_len(p)):
            for y in len:
                try:
                    label_b = label[i][1][r][:y]
                    label_b = list(label_b)
                    while get_len(label_b) < y:
                        label_b += '0'
                    label_b = ''.join(label_b)
                    if '1' in label_b:
                        label_dict['label_r_p'.replace('r',str(p[r])).replace('p',str(y))].append([label[i][0][r][:y],label_b])
                except:
                    pass
                    # print('error')
                # print(label_dict['label_r_p'.replace('r',str(p[r])).replace('p',str(y))])

    for i in label_dict.keys():
        with open(file_name+'/'+i+'_'+pos_tag+'.csv',mode='w') as f:
            wt = csv.writer(f)
            for r in label_dict[i]:
                wt.writerow([r[0]]+list(r[1]))
    
    # save mask
    for k in range(get_len(mask)):
        for i in len:
            with open(file_name+'/'+'mask_'+str(i)+'.csv',mode='a') as f:
                wt = csv.writer(f)
                if 'MASK' in mask[k][0][:i]:
                    wt.writerow([mask[k][0][:i],mask[k][1][:i]])

word_data(['傅达仁今将运行安乐死，突然爆出自己20年前遭纬来体育台封杀，他不懂自己哪里得罪到电视台。'], [64,32], [0.5,0.8], 'D')

In [ ]:
lengths = [96,72,64,32,16]
ps = [0.8,0.5,0.2]
tags = ['A','D','P','Nf','Cbb']

for i in tags:
    word_data(sents,lengths,ps,i)

Chenyu utils

In [2]:
chenyu_file = 'chenyu.csv'
chenyus = []
sents_ = []

with open(chenyu_file,'r') as f:
    x = csv.reader(f)
    chenyus = [i[0] for i in x][1:]

with open(chenyu_file,'r') as f:
    x = csv.reader(f)
    sents_ = [i[1] for i in x][1:]

sents = sents_

In [6]:
def get_len(a):
    return(len(a))

def chenyu_data(sents:list, chenyus:list, len:list, p_:list):
    file_name = 'chenyu_'+str(get_len(sents))
    if not os.path.isdir(file_name):
        os.mkdir(file_name)

    sents_ = []
    mask = []
    label = []

    for p in p_:

        for sent in tqdm(range(get_len(sents))):
            try:
                chenyu_ = chenyus[sent]
                sent = sents[sent][:max(len)]
                a, b= chenyu(sent, chenyu_, p)
                label.append(b)
                mask += [a]
                del a
                del b
            except:
                pass

    # save dict
    label_dict = {}
    for y in len:
        label_dict['label_r_p'.replace('r',str(p)).replace('p',str(y))] = []

        for x in range(get_len(label)):
            label[x][1] = list(label[x][1])[:max(len)]
            while get_len(label[x][1]) < y:
                label[x][1] += '0'
            label[x][1] = ''.join(label[x][1][:y])

            label_dict['label_r_p'.replace('r',str(p)).replace('p',str(y))].append(label[x])
        
        dict_to_save = 'label_r_p'.replace('r',str(p)).replace('p',str(y))
        with open(file_name+'/'+dict_to_save+'.csv',mode='a') as f:
            wt = csv.writer(f)
            for r in label_dict[dict_to_save]:
                wt.writerow([r[0]]+list(r[1]))
    
    # save mask
    for k in range(get_len(mask)):
        for i in len:
            with open(file_name+'/'+'mask_'+str(i)+'.csv',mode='a') as f:
                wt = csv.writer(f)
                count = 0
                for t in mask[k][0]:
                    if t == 'M':count+=1
                if count <=1 and get_len(mask[k][0]) <= i and get_len(mask[k][1]) <= i:
                    wt.writerow([mask[k][0],mask[k][1]])

chenyu_data(['细致地挨门逐户去调查访问。'], ['挨门逐户'], [64,32], [0.8])

100%|██████████| 1/1 [00:00<00:00, 2344.50it/s]


In [4]:
lengths = [96,72,64,32,16]
ps = [0.8,0.5,0.2]

for i in ps:
    chenyu_data(sents, chenyus,lengths,[i])

    

100%|██████████| 51657/51657 [00:06<00:00, 7577.66it/s]


In [ ]:
'''
不太政治正確的例句:
    干革命不只是MASK重要的是为了争得全体劳苦大众的解放
    李登辉的险恶用心现在已经暴露无遗了
    人民安居乐业，这就是我亲爱的祖国
    新中国成立后，中国人民安居乐业，过上了好日子
    中国人民万众一心打败了美帝国主义。
'''